# Baseline

In [ ]:
# Install required packages
%pip install --upgrade pip
%pip install polars numpy scikit-learn matplotlib joblib openpyxl fastexcel tensorflow

# For TensorFlow on Mac, you need to install tensorflow-macos
%pip install tensorflow-macos tensorflow-metal

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement tensorflow-macos (from versions: none)
ERROR: No matching distribution found for tensorflow-macos
Note: you may need to restart the kernel to use updated packages.


Importación de las librerías a usar.

In [2]:
import polars as pl
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import matplotlib.pyplot as plt
import os
from joblib import Parallel, delayed
from datetime import timedelta
import openpyxl

# Configuración de Matplotlib para evitar errores con Tkinter
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

2025-04-01 21:22:21.925 Python[81840:81126064] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/b9/dh9qy_tj6p7g8mrkr5cj9rmc0000gn/T/org.python.python.savedState


## Definición de las rutas y sujetos

In [4]:
# Obtener los archivos de los sujetos
# Definición de la ruta del proyecto
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Definición de la ruta de los archivos de los sujetos
SUBJECTS_RELATIVE_PATH = "data/Subjects"
SUBJECTS_PATH = os.path.join(PROJECT_ROOT, SUBJECTS_RELATIVE_PATH)

# Definición de la ruta de salida de las figuras
FIGURES_DIR = os.path.join(PROJECT_ROOT, "figures", "baseline")
os.makedirs(FIGURES_DIR, exist_ok=True)

subject_files = [f for f in os.listdir(SUBJECTS_PATH) if f.startswith("Subject") and f.endswith(".xlsx")]
print(f"Total sujetos: {len(subject_files)}")

Total sujetos: 54


## Paso 1: Preprocesamiento.

Leer los datos de los sujetos y preprocesarlos para el modelo, seleccionando las columnas de interés.

### Data Preprocessing Functions

Definición de las funciones:

- `get_cgm_window`: obtiene la ventana de datos de CGM para un tiempo de bolo específico.
- `calculate_iob`: calcula la insulina activa en el cuerpo para un tiempo de bolo específico.
- `process_subject`: procesa los datos de un sujeto específico.

In [4]:
def get_cgm_window(bolus_time, cgm_df, window_hours=2) -> np.ndarray:
    '''
    Obtiene la ventana de datos de CGM para un tiempo de bolo específico.

    Parámetros:
    - bolus_time: datetime
        Tiempo del bolo de insulina.
    - cgm_df: pl.DataFrame
        Datos de CGM.
    - window_hours: int
        Número de horas de la ventana de datos.
    
    Retorna:
    - np.ndarray
        Ventana de datos
    '''
    window_start = bolus_time - timedelta(hours=window_hours)
    # Filtro y ordenamiento de los datos de CGM
    window = cgm_df.filter(
        (pl.col("date") >= window_start) & (pl.col("date") <= bolus_time)
    ).sort("date").tail(24)
    
    if window.height < 24:
        return None
    return window.get_column("mg/dl").to_numpy()

def calculate_iob(bolus_time, basal_df, half_life_hours=4) -> float:
    '''
    Calcula la insulina activa en el cuerpo (IOB) para un tiempo de bolo específico.

    Parámetros:
    - bolus_time: datetime
        Tiempo del bolo de insulina.
    - basal_df: pl.DataFrame
        Datos de insulina basal.
    - half_life_hours: float
        Vida media de la insulina en horas.
    
    Retorna:
    - float
        Insulina activa en el cuerpo.
    '''
    if basal_df is None or basal_df.is_empty():
        return 0.0
    
    iob = 0
    for row in basal_df.iter_rows(named=True):
        start_time = row["date"]
        duration_hours = row["duration"] / (1000 * 3600)
        end_time = start_time + timedelta(hours=duration_hours)
        rate = row["rate"] if row["rate"] is not None else 0.9
        
        if start_time <= bolus_time <= end_time:
            time_since_start = (bolus_time - start_time).total_seconds() / 3600
            remaining = rate * (1 - (time_since_start / half_life_hours))
            iob += max(0, remaining)
    return iob

def process_subject(subject_path, idx) -> list:
    '''
    Procesa los datos de un sujeto específico.

    Parámetros:
    - subject_path: str
        Ruta del archivo de datos del sujeto.
    - idx: int
        Índice del sujeto.
    
    Retorna:
    - list
        Datos procesados del sujeto.
    '''
    print(f"Procesando {os.path.basename(subject_path)} ({idx+1}/{len(subject_files)})...")
    
    try:
        # Carga de los datos del sujeto
        cgm_df = pl.read_excel(subject_path, sheet_name="CGM")
        bolus_df = pl.read_excel(subject_path, sheet_name="Bolus")
        try:
            basal_df = pl.read_excel(subject_path, sheet_name="Basal")
        except Exception:
            basal_df = None
            
    except Exception as e:
        print(f"Error loading {os.path.basename(subject_path)}: {e}")
        return []

    # Conversión de fechas y preordenamiento CGM para cada sujeto
    cgm_df = cgm_df.with_columns(pl.col("date").cast(pl.Datetime))
    bolus_df = bolus_df.with_columns(pl.col("date").cast(pl.Datetime))
    if basal_df is not None:
        basal_df = basal_df.with_columns(pl.col("date").cast(pl.Datetime))
    
    # Preordenamiento para eficiencia
    cgm_df = cgm_df.sort("date")

    processed_data = []
    for row in bolus_df.iter_rows(named=True):
        bolus_time = row["date"]
        cgm_window = get_cgm_window(bolus_time, cgm_df)
        
        if cgm_window is not None:
            iob = calculate_iob(bolus_time, basal_df)
            features = {
                'subject_id': idx,
                'cgm_window': cgm_window,
                'carbInput': row["carbInput"] if row["carbInput"] is not None else 0.0,
                'bgInput': row["bgInput"] if row["bgInput"] is not None else cgm_window[-1],
                'insulinCarbRatio': row["insulinCarbRatio"] if row["insulinCarbRatio"] is not None else 10.0,
                'insulinSensitivityFactor': 50.0,
                'insulinOnBoard': iob,
                'normal': row["normal"]
            }
            processed_data.append(features)
    
    return processed_data

# Ejecución en paralelo
all_processed_data = Parallel(n_jobs=-1)(
    delayed(process_subject)(
        os.path.join(SUBJECTS_PATH, f), 
        idx
    ) for idx, f in enumerate(subject_files)
)

all_processed_data = [item for sublist in all_processed_data for item in sublist]

# Conversión a DataFrame
df_processed = pl.DataFrame(all_processed_data)
print("Muestra de datos procesados combinados:")
print(df_processed.head())
print(f"Total muestras: {len(df_processed)}")

Procesando Subject21.xlsx (1/54)...
Procesando Subject37.xlsx (2/54)...
Procesando Subject17.xlsx (3/54)...
Procesando Subject40.xlsx (4/54)...
Procesando Subject6.xlsx (5/54)...
Procesando Subject7.xlsx (6/54)...
Procesando Subject41.xlsx (7/54)...
Procesando Subject16.xlsx (8/54)...
Procesando Subject36.xlsx (9/54)...
Procesando Subject20.xlsx (10/54)...
Procesando Subject11.xlsx (11/54)...
Procesando Subject46.xlsx (12/54)...
Procesando Subject50.xlsx (13/54)...
Procesando Subject27.xlsx (14/54)...
Procesando Subject31.xlsx (15/54)...
Procesando Subject30.xlsx (16/54)...
Procesando Subject26.xlsx (17/54)...
Procesando Subject1.xlsx (18/54)...
Procesando Subject51.xlsx (19/54)...
Procesando Subject47.xlsx (20/54)...
Procesando Subject10.xlsx (21/54)...
Procesando Subject29.xlsx (22/54)...
Procesando Subject2.xlsx (23/54)...
Procesando Subject52.xlsx (24/54)...
Procesando Subject44.xlsx (25/54)...
Procesando Subject13.xlsx (26/54)...


Could not determine dtype for column 5, falling back to string


Procesando Subject33.xlsx (27/54)...
Procesando Subject25.xlsx (28/54)...
Procesando Subject48.xlsx (29/54)...
Procesando Subject49.xlsx (30/54)...
Procesando Subject24.xlsx (31/54)...
Procesando Subject32.xlsx (32/54)...
Procesando Subject12.xlsx (33/54)...
Procesando Subject45.xlsx (34/54)...
Procesando Subject53.xlsx (35/54)...
Procesando Subject3.xlsx (36/54)...
Procesando Subject28.xlsx (37/54)...
Procesando Subject35.xlsx (38/54)...
Procesando Subject23.xlsx (39/54)...
Procesando Subject8.xlsx (40/54)...
Procesando Subject19.xlsx (41/54)...
Procesando Subject39.xlsx (42/54)...
Procesando Subject4.xlsx (43/54)...
Procesando Subject54.xlsx (44/54)...
Procesando Subject42.xlsx (45/54)...
Procesando Subject15.xlsx (46/54)...
Procesando Subject14.xlsx (47/54)...
Procesando Subject43.xlsx (48/54)...
Procesando Subject5.xlsx (49/54)...
Procesando Subject38.xlsx (50/54)...
Procesando Subject18.xlsx (51/54)...
Procesando Subject9.xlsx (52/54)...
Procesando Subject22.xlsx (53/54)...
Proces

### División y Combinación de Datos Procesados

In [5]:
# División de la ventana CGM en columnas
cgm_columns = [f'cgm_{i}' for i in range(24)]
df_cgm = pl.DataFrame({
    col: [row['cgm_window'][i] for row in all_processed_data]
    for i, col in enumerate(cgm_columns)
},
schema={col: pl.Float64 for col in cgm_columns})

# Combinación con los datos procesados
df_final = pl.concat([
    df_cgm,
    df_processed.drop('cgm_window')
], how="horizontal")

### Eliminación de valores nulos.

In [6]:
# Eliminar valores nulos en df_final.
print("Check de valores nulos en df_final:")
df_final = df_final.drop_nulls()
print(df_final.null_count())

Check de valores nulos en df_final:
shape: (1, 31)
┌───────┬───────┬───────┬───────┬───┬──────────────────┬─────────────────┬────────────────┬────────┐
│ cgm_0 ┆ cgm_1 ┆ cgm_2 ┆ cgm_3 ┆ … ┆ insulinCarbRatio ┆ insulinSensitiv ┆ insulinOnBoard ┆ normal │
│ ---   ┆ ---   ┆ ---   ┆ ---   ┆   ┆ ---              ┆ ityFactor       ┆ ---            ┆ ---    │
│ u32   ┆ u32   ┆ u32   ┆ u32   ┆   ┆ u32              ┆ ---             ┆ u32            ┆ u32    │
│       ┆       ┆       ┆       ┆   ┆                  ┆ u32             ┆                ┆        │
╞═══════╪═══════╪═══════╪═══════╪═══╪══════════════════╪═════════════════╪════════════════╪════════╡
│ 0     ┆ 0     ┆ 0     ┆ 0     ┆ … ┆ 0                ┆ 0               ┆ 0              ┆ 0      │
└───────┴───────┴───────┴───────┴───┴──────────────────┴─────────────────┴────────────────┴────────┘


## Paso 2: Entrenamiento del modelo.
Normalización de los datos y división en conjuntos de entrenamiento, validación y prueba.

In [7]:
scaler_cgm = MinMaxScaler(feature_range=(0, 1))
scaler_other = StandardScaler()

# Normalizar características
X_cgm = scaler_cgm.fit_transform(df_final.select(cgm_columns).to_numpy())
X_other = scaler_other.fit_transform(
    df_final.select(['carbInput', 'bgInput', 'insulinOnBoard']).to_numpy()
)

# Combinar características
X = np.hstack([
    X_cgm, 
    X_other, 
    df_final.select(['insulinCarbRatio', 'insulinSensitivityFactor', 'subject_id']).to_numpy()
])
y = df_final.get_column('normal').to_numpy()

In [8]:
# Conversión de arrays a NumPy
X = np.asarray(X)
y = np.asarray(y)

print("X dtype:", X.dtype)
print("y dtype:", y.dtype)

def check_nan(arr) -> int:
    '''
    Función para verificar valores NaN en un array NumPy.

    Parámetros:
    - arr: np.ndarray
        Array NumPy a verificar.

    Retorna:
    - int
        Número de valores NaN en el array.
    '''
    if np.issubdtype(arr.dtype, np.number):
        return np.isnan(arr).sum()
    else:
        return np.sum([x is None for x in arr])

print("NaN in X:", check_nan(X))
print("NaN in y:", check_nan(y))

# Levantar error si hay valores en NaN
if check_nan(X) > 0 or check_nan(y) > 0:
    raise ValueError("Valores NaN detectados en X o y")

X dtype: float64
y dtype: float64
NaN in X: 0
NaN in y: 0


### División de Datos por Sujeto

In [9]:
subject_ids = df_final.get_column('subject_id').unique().to_numpy()
train_subjects, temp_subjects = train_test_split(subject_ids, test_size=0.2, random_state=42)
val_subjects, test_subjects = train_test_split(temp_subjects, test_size=0.5, random_state=42)

train_mask = np.isin(df_final.get_column('subject_id').to_numpy(), train_subjects)
val_mask = np.isin(df_final.get_column('subject_id').to_numpy(), val_subjects)
test_mask = np.isin(df_final.get_column('subject_id').to_numpy(), test_subjects)

X_train, y_train = X[train_mask], y[train_mask]
X_val, y_val = X[val_mask], y[val_mask]
X_test, y_test = X[test_mask], y[test_mask]
subject_test = df_final.filter(pl.lit(test_mask)).get_column('subject_id').to_numpy()

print(f"Entrenamiento: {X_train.shape}, Validación: {X_val.shape}, Test: {X_test.shape}")
print(f"Sujetos de prueba: {test_subjects}")

Entrenamiento: (33272, 30), Validación: (2742, 30), Test: (8607, 30)
Sujetos de prueba: [ 5 19 32 13 48 49]


## Paso 3: Establecimiento del ;odelo Base y Entrenamiento.

Definición del modelo FNN (Feedforward Neural Network), ajustado para un conjunto más grande de datos.

Entrenamiento del modelo.

In [10]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='linear')
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')
model.summary()

# Convertir datos to float32
X_train = np.array(X_train, dtype=np.float32)
X_val = np.array(X_val, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
y_val = np.array(y_val, dtype=np.float32)

# Verificar formas y tipos de datos.
DTYPE_TEXT = "dtype:"
print("X_train shape:", X_train.shape, DTYPE_TEXT, X_train.dtype)
print("X_val shape:", X_val.shape, DTYPE_TEXT, X_val.dtype)
print("y_train shape:", y_train.shape, DTYPE_TEXT, y_train.dtype)
print("y_val shape:", y_val.shape, DTYPE_TEXT, y_val.dtype)

# Entrenar el modelo.
print("\nEntrenando el modelo...")
EPOCHS = 100
BATCH_SIZE = 32
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)],
    verbose=1
)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         1,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,097 (16.00 KB)

 Trainable params: 4,097 (16.00 KB)

 Non-trainable params: 0 (0.00 B)

X_train shape: (33272, 30) dtype: float32
X_val shape: (2742, 30) dtype: float32
y_train shape: (33272,) dtype: float32
y_val shape: (2742,) dtype: float32

Entrenando el modelo...
Epoch 1/100
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 18.5062 - val_loss: 2.3765
Epoch 2/100
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 6.5213 - val_loss: 1.4072
Epoch 3/100
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 5.7208 - val_loss: 1.4470
Epoch 4/100
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 5.3142 - val_loss: 1.2671
Epoch 5/100
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 5.3252 - val_loss: 1.0730
Epoch 6/100
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 4.6771 - val_loss: 1.1519
Epoch 7/100
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 4.4172 - val_loss: 1.0694
Epoch 8/100
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 4.3281 - val_loss: 1.1659
Epoch 9/100
1040/1040 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 4.1560 - val_loss: 1.1901
Epoch 10/100
1040

## Paso 4: Graficación de los datos.

Graficar el historial de entrenamiento.

In [11]:
figure_path = os.path.join(FIGURES_DIR, 'evolucion.png')

plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Pérdida del Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de la Validación')
plt.xlabel('Épocas/Epochs')
plt.ylabel('Pérdida ECM/MSE Loss')
plt.legend()
plt.title('Historial de Entrenamiento (todos los sujetos)')
plt.savefig(figure_path, dpi=300, bbox_inches='tight')

print(f"Figura guardada en: {figure_path}")

Figura guardada en: /Users/tomas/Desktop/FIUBA/TPP/TPP_Deep_Learning_Models/figures/baseline/evolucion.png


## Paso 5: Evaluación del Modelo Base.

Evaluar la performance del modelo según las métricas MAE, RMSE y R².

In [12]:
y_pred = model.predict(X_test).flatten()

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"General MAE (Mean Absolute Error): {mae:.2f} units")
print(f"General RMSE (Root Meean Squared Error): {rmse:.2f} units")
print(f"General R²: {r2:.2f}")

269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
General MAE (Mean Absolute Error): 1.16 units
General RMSE (Root Meean Squared Error): 1.62 units
General R²: 0.54


### Limpieza de X_test

In [13]:
def clean_array(arr: np.ndarray, return_counts: bool = False) -> tuple[np.ndarray, dict] | np.ndarray:
    """
    Limpia un array numpy eliminando valores infinitos y demasiado grandes para float64.
    
    Parámeteros:
    -----------
    arr : np.ndarray
        Array a limpiar
    return_counts : bool, optional
        Si es True, se devolverá un diccionario con los conteos de valores eliminados
        
    Retorna:
    --------
    np.ndarray o tuple[np.ndarray, dict]
        Array limpio o tupla con array limpio y diccionario
    """
    # Verificar dimensiones del array
    if arr.ndim != 2:
        raise ValueError(f"El array debe ser 2D. Forma actual: {arr.shape}")
    
    max_float = np.finfo(np.float64).max
    min_float = np.finfo(np.float64).min
    
    # Crear máscaras para valores problemáticos
    inf_mask = np.isinf(arr)
    too_large_mask = (arr > max_float) | (arr < min_float)
    nan_mask = np.isnan(arr)
    
    # Combinar máscaras
    problem_mask = inf_mask | too_large_mask | nan_mask
    # Convertir a máscara por filas
    row_mask = ~np.any(problem_mask, axis=1)
    
    # Contar valores problemáticos
    counts = {
        'inf_values': np.sum(inf_mask),
        'too_large_values': np.sum(too_large_mask & ~inf_mask),
        'nan_values': np.sum(nan_mask),
        'total_rows_removed': np.sum(~row_mask)
    }
    
    # Limpiar array manteniendo la forma 2D
    cleaned_arr = arr[row_mask]
    
    if return_counts:
        return cleaned_arr, counts
    return cleaned_arr

X_test_cleaned, counts = clean_array(X_test, return_counts=True)
print(f"Resumen de valores eliminados:")
for k, v in counts.items():
    print(f"{k}: {v}")

Resumen de valores eliminados:
inf_values: 0
too_large_values: 0
nan_values: 0
total_rows_removed: 0


### Línea base basada en reglas.

In [14]:
# Línea base basada en reglas.
def rule_based_prediction(arr: np.ndarray, target_bg: float = 100.0) -> np.ndarray:
    """
    Función para predecir la dosis de insulina basada en reglas.
    
    Parámetros:
    -----------
    arr : np.ndarray
        Array 2D de entrada con features
    target_bg : float
        Valor objetivo de glucosa en sangre
        
    Retorna:
    --------
    np.ndarray
        Predicciones de dosis de insulina
    """
    # Verificar dimensiones
    if arr.ndim != 2:
        raise ValueError(f"El array debe ser 2D. Forma actual: {arr.shape}")
    
    if arr.shape[1] < 29:
        raise ValueError(f"El array debe tener al menos 29 columnas. Actuales: {arr.shape[1]}")
    
    # Transformar datos una sola vez
    transformed_data = scaler_other.inverse_transform(arr[:, 24:27])
    
    # Extraer features
    carb_input = transformed_data[:, 0]
    bg_input = transformed_data[:, 1]
    icr = arr[:, 27]
    isf = arr[:, 28]
    
    # Crear máscara para valores válidos
    valid_mask = (icr != 0) & (isf != 0)
    
    # Inicializar array de predicciones con NaN
    predictions = np.full(len(arr), np.nan)
    
    # Calcular predicciones solo donde ICR e ISF son válidos
    predictions[valid_mask] = (
        carb_input[valid_mask] / icr[valid_mask] + 
        (bg_input[valid_mask] - target_bg) / isf[valid_mask]
    )
    
    # Casos inválidos:
    if not np.all(valid_mask):
        print(f"Advertencia: {np.sum(~valid_mask)} predicciones no pudieron calcularse debido a ICR o ISF igual a cero")
    
    return predictions

try:
    y_rule = rule_based_prediction(X_test_cleaned)
    mae_rule = mean_absolute_error(y_test, y_rule)
    rmse_rule = np.sqrt(mean_squared_error(y_test, y_rule))
    r2_rule = r2_score(y_test, y_rule)
    print(f"MAE basado en reglas: {mae_rule:.2f} u. de insulina.")
    print(f"RMSE basado en reglas: {rmse_rule:.2f} u. de insulina.")
    print(f"R² basado en reglas: {r2_rule:.2f}")
except ValueError as e:
    print(f"Error en la predicción basada en reglas: {str(e)}")
    print("\nDetalles de los datos:")
    print(f"Shape de X_test: {X_test.shape}")
    print(f"NaN en X_test: {np.isnan(X_test).sum()}")
    print(f"NaN en y_test: {np.isnan(y_test).sum()}")

Advertencia: 1165 predicciones no pudieron calcularse debido a ICR o ISF igual a cero
Error en la predicción basada en reglas: Input contains NaN.

Detalles de los datos:
Shape de X_test: (8607, 30)
NaN en X_test: 0
NaN en y_test: 0


### Métricas por Sujeto

In [15]:
print("\nRendimiento por sujeto:")
for subject_id in test_subjects:
    mask = subject_test == subject_id
    y_test_sub = y_test[mask]
    y_pred_sub = y_pred[mask]
    
    # Clean data for subject-wise metrics
    sub_data = np.column_stack([y_test_sub, y_pred_sub])
    is_finite = np.all(np.isfinite(sub_data), axis=1)
    
    if len(y_test_sub) > 0 and np.any(is_finite):
        # Use only finite values for metrics
        y_test_clean = y_test_sub[is_finite]
        y_pred_clean = y_pred_sub[is_finite]
        
        # Calculate metrics
        mae_sub = mean_absolute_error(y_test_clean, y_pred_clean)
        rmse_sub = np.sqrt(mean_squared_error(y_test_clean, y_pred_clean))
        r2_sub = r2_score(y_test_clean, y_pred_clean)
        
        # Rule-based metrics if available
        try:
            y_rule_sub = y_rule[mask][is_finite]
            mae_rule_sub = mean_absolute_error(y_test_clean, y_rule_sub)
            print(
                f"Sujeto {subject_id}: "
                f"FNN MAE={mae_sub:.2f}, "
                f"RMSE={rmse_sub:.2f}, "
                f"R²={r2_sub:.2f}, "
                f"MAE basado en reglas={mae_rule_sub:.2f}"
            )
        except (IndexError, ValueError):
            print(
                f"Sujeto {subject_id}: "
                f"FNN MAE={mae_sub:.2f}, "
                f"RMSE={rmse_sub:.2f}, "
                f"R²={r2_sub:.2f}, "
                f"MAE basado en reglas=N/A"
            )
    else:
        print(f"Sujeto {subject_id}: No hay suficientes datos válidos para calcular métricas")


Rendimiento por sujeto:
Sujeto 5: FNN MAE=1.24, RMSE=1.52, R²=-0.38, MAE basado en reglas=1.87
Sujeto 19: FNN MAE=1.75, RMSE=2.07, R²=0.32, MAE basado en reglas=1.25
Sujeto 32: FNN MAE=0.81, RMSE=1.18, R²=0.72, MAE basado en reglas=1.73
Sujeto 13: FNN MAE=0.92, RMSE=1.05, R²=-0.52, MAE basado en reglas=1.17
Sujeto 48: FNN MAE=1.06, RMSE=1.35, R²=0.12, MAE basado en reglas=N/A
Sujeto 49: FNN MAE=2.38, RMSE=3.55, R²=0.37, MAE basado en reglas=2.14


### Visualización

#### Separación de Datos

Separación entre datos limpios y residuales.

In [16]:
def clean_residuals(actual, predicted):
    """
    Limpia los residuos eliminando valores infinitos.
    
    Retorna:
    --------
    - np.ndarray
        Residuos limpios
    - int
        Número de valores eliminados
    """
    residuals = actual - predicted
    mask = np.isfinite(residuals)
    n_dropped = np.sum(~mask)
    return residuals[mask], n_dropped

fnn_residuals, fnn_dropped = clean_residuals(y_test, y_pred)
rule_residuals, rule_dropped = clean_residuals(y_test, y_rule)

print(f"\nValores infinitos removidos:")
print(f"FNN: {fnn_dropped} valores")
print(f"Basado en reglas: {rule_dropped} valores")

plt.figure(figsize=(12, 5))


Valores infinitos removidos:
FNN: 0 valores
Basado en reglas: 1165 valores


<Figure size 1200x500 with 0 Axes>

#### Predicción vs Real

In [17]:
plt.subplot(1, 2, 1)
mask_pred = np.isfinite(y_pred)
mask_rule = np.isfinite(y_rule)

plt.scatter(y_test[mask_pred], y_pred[mask_pred], label='Predicción FNN', alpha=0.5)
plt.scatter(y_test[mask_rule], y_rule[mask_rule], label='Basado en Reglas', alpha=0.5)
plt.plot([0, 15], [0, 15], 'r--')
plt.xlabel('Dosis Real (u. de insulina)')
plt.ylabel('Dosis Predicha (u. de insulina)')
plt.legend()
plt.title('Predicción vs. Real (Todos los Sujetos)')

Text(0.5, 1.0, 'Predicción vs. Real (Todos los Sujetos)')

#### Residuales

In [18]:
plt.subplot(1, 2, 2)
plt.hist(fnn_residuals, bins=20, label='Residuales FNN', alpha=0.5)
plt.hist(rule_residuals, bins=20, label='Residuos Basados en Reglas', alpha=0.5)
plt.xlabel('Residuo (u. de insulina)')
plt.ylabel('Frecuencia')
plt.legend()
plt.title('Distribución de Residuos (Todos los Sujetos)')

Text(0.5, 1.0, 'Distribución de Residuos (Todos los Sujetos)')

In [ ]:
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, 'pred_vs_real.png'), dpi=300, bbox_inches='tight')
plt.close()

: 